In [ ]:
!pip install Bio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.4/276.4 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 40.8 MB/s eta 0:00:00


In [ ]:
## this little script shows off the use of the pubmed API through bioconductor
## requires installing Biopython (using pip)
## also requires installing the DTD files for each of the Entrez API calls,
## but the instructions for this are given when you run the script

## useful list of Entrez databases that can be queried through API
# pmc_pubmed	PubMed citations for these articles
# pmc_refs_pubmed	PubMed article citing PMC article
# pmc_pmc_cites	   PMC articles that given PMC article cites
# pmc_pmc_citedby	PMC article citing given PMC article
# pubmed_pubmed	 Calculated set of PubMed citations similar to the selected article(s) retrieved using a word weight algorithm.
# pubmed_pubmed_refs    Citation referenced in PubMed article. Only valid for PubMed citations that are also in PMC.


from Bio import Entrez

Entrez.email = "mcfrank@stanford.edu"

def get_abstract(pmid):
    handle = Entrez.efetch(db='pubmed', id=pmid, retmode='text', rettype='abstract')
    return handle.read()

def get_links_id(pmid):
	link_list = []
	links = Entrez.elink(dbfrom="pubmed", id=pmid, linkname="pubmed_pubmed")
	record = Entrez.read(links)

	records = record[0][u'LinkSetDb'][0][u'Link']

	for link in records:
		link_list.append(link[u'Id'])

	return link_list

def get_links_term(term):
	links = Entrez.esearch(db="pubmed", retmax = 10000, term=term) # max retrieve is 9999
	record = Entrez.read(links)
	link_list = record[u'IdList']

	return link_list


### MAIN -----------------------
links_term = list()
links_term = get_links_term('"2023/08/10"[PDAT] : "2023/08/12"[PDAT]') #example of term: Saffran JR[Author], drug combination, 0000/01/01"[PDAT] : "3000/12/31"[PDAT] AND drug combination
print(links_term)

# print("----------------------------")

# print(get_abstract("35043159"))

# print("----------------------------")

# links_id = get_links_id("35043159")
# print(links_id) # similar articles

['37571984', '37571983', '37571982', '37571981', '37571980', '37571979', '37571978', '37571977', '37571976', '37571975', '37571974', '37571973', '37571972', '37571971', '37571970', '37571969', '37571968', '37571967', '37571966', '37571964', '37571963', '37571962', '37571961', '37571960', '37571959', '37571958', '37571957', '37571956', '37571931', '37571930', '37571929', '37571928', '37571927', '37571926', '37571925', '37571924', '37571923', '37571922', '37571921', '37571920', '37571919', '37571918', '37571917', '37571916', '37571915', '37571914', '37571913', '37571912', '37571911', '37571910', '37571878', '37571877', '37571876', '37571875', '37571874', '37571873', '37571872', '37571871', '37571870', '37571869', '37571868', '37571867', '37571866', '37571865', '37571864', '37571863', '37571862', '37571861', '37571860', '37571859', '37571858', '37571857', '37571856', '37571855', '37571854', '37571853', '37571852', '37571851', '37571850', '37571849', '37571846', '37571845', '37571843', '37

In [ ]:
print(links_term[0])
print(links_term[-1])

37571984
36518094


In [ ]:
import pandas as pd
import time

start_time = time.time()

def extract_parts(string):
  str_split = string.split("\n\n")

  return str_split[1], str_split[4] # 1 for title and 4 for abstract


articles_raw = []
test = links_term[:10]
for id in test:
  try:
    string = get_abstract(id)
    title, abstract = extract_parts(string)
    if not abstract.startswith('DOI'): # there is no abstract available
      articles_raw.append([id, title, abstract])
  except:
    print(f"article {id} is not available")

print("--- %s seconds ---" % (time.time() - start_time))
articles_raw

article 37571787 is not available
--- 6.6954851150512695 seconds ---


[['37571796',
  'Unveiling the Dynamics of Thermal Characteristics Related to LULC Changes via \nANN.',
  "Continuous and unplanned urbanization, combined with negative alterations in \nland use land cover (LULC), leads to a deterioration of the urban thermal \nenvironment and results in various adverse ecological effects. The changes in \nLULC and thermal characteristics have significant implications for the economy, \nclimate patterns, and environmental sustainability. This study focuses on the \nProvince of Naples in Italy, examining LULC changes and the Urban Thermal Field \nVariance Index (UTFVI) from 1990 to 2022, predicting their distributions for \n2030. The main objectives of this research are the investigation of the future \nseasonal thermal characteristics of the study area by characterizing land \nsurface temperature (LST) through the UTFVI and analyzing LULC dynamics along \nwith their correlation. To achieve this, Landsat 4-5 Thematic Mapper (TM) and \nLandsat 9 Operatio

In [ ]:
articles = pd.DataFrame(articles_raw, columns=['id', 'title', 'abstract'])
articles

,id,title,abstract
0,37565463,Does Referral Distance Deteriorates the Burn P...,Every year millions of people are burned and m...
1,37565461,Modification of Branched poly(Ethylene Imine) ...,Branched poly(ethylene imine) (bPEI) is freque...
2,37565458,Efficacy and safety of vonoprazan-based dual t...,BACKGROUND: According to the Maastricht VI/Flo...
3,37565457,Acute Effect of Heavy Weightlifting on the Pel...,INTRODUCTION/PURPOSE: Heavy lifting may produc...
4,37565455,Biochemical Resistivity against Free Radicals ...,This work delivers a targeted synthesis of fou...
...,...,...,...
7669,36350716,Creating an automated contemporaneous cohort i...,The Food and Drug Administration requires cont...
7670,36322827,miR-148a-3p and DDX6 functional link promotes ...,Regulation of gene expression at the RNA level...
7671,36240477,Analysis of disparities in time to allogeneic ...,Although alternative donors extend transplant ...
7672,36129842,Hodgkin lymphoma and female fertility: a multi...,Preservation of fertility has become a growing...


In [ ]:
articles.to_csv("articles.tsv", sep='\t')

In [ ]:
print(get_abstract('37565456'))

1. J Eur Acad Dermatol Venereol. 2023 Aug 10. doi: 10.1111/jdv.19405. Online
ahead  of print.

Risk of HPV infection among dermatologists: Are we prepared?

Leahy M(#)(1), Harrington R(#)(1), Laing M(1)(2).

Author information:
(1)Department of Dermatology, University Hospital Galway, Galway, Ireland.
(2)School of medicine, National University of Ireland Galway.
(#)Contributed equally

DOI: 10.1111/jdv.19405
PMID: 37565456


In [ ]:
print(get_abstract("35043159"))

1. Brief Bioinform. 2022 Mar 10;23(2):bbab587. doi: 10.1093/bib/bbab587.

PRODeepSyn: predicting anticancer synergistic drug combinations by embedding 
cell lines with protein-protein interaction network.

Wang X(1), Zhu H(1), Jiang Y(1), Li Y(1), Tang C(1), Chen X(1), Li Y(1), Liu 
Q(2), Liu Q(1).

Author information:
(1)School of Software Engineering, Tongji University, Shanghai, China.
(2)Translational Medical Center for Stem Cell Therapy and Institute for 
Regenerative Medicine, Shanghai East Hospital, Shanghai Key Laboratory of 
Signaling and Disease Research, School of Life Sciences and Technology, Tongji 
University, Shanghai, China.

Although drug combinations in cancer treatment appear to be a promising 
therapeutic strategy with respect to monotherapy, it is arduous to discover new 
synergistic drug combinations due to the combinatorial explosion. Deep learning 
technology holds immense promise for better prediction of in vitro synergistic 
drug combinations for certain cell 

In [ ]:
df = pd.read_csv('articles.tsv', sep='\t')
df

,Unnamed: 0,id,title,abstract
0,0,37565463,Does Referral Distance Deteriorates the Burn P...,Every year millions of people are burned and m...
1,1,37565461,Modification of Branched poly(Ethylene Imine) ...,Branched poly(ethylene imine) (bPEI) is freque...
2,2,37565458,Efficacy and safety of vonoprazan-based dual t...,BACKGROUND: According to the Maastricht VI/Flo...
3,3,37565457,Acute Effect of Heavy Weightlifting on the Pel...,INTRODUCTION/PURPOSE: Heavy lifting may produc...
4,4,37565455,Biochemical Resistivity against Free Radicals ...,This work delivers a targeted synthesis of fou...
...,...,...,...,...
7669,7669,36350716,Creating an automated contemporaneous cohort i...,The Food and Drug Administration requires cont...
7670,7670,36322827,miR-148a-3p and DDX6 functional link promotes ...,Regulation of gene expression at the RNA level...
7671,7671,36240477,Analysis of disparities in time to allogeneic ...,Although alternative donors extend transplant ...
7672,7672,36129842,Hodgkin lymphoma and female fertility: a multi...,Preservation of fertility has become a growing...


### Working with EDirect

In [ ]:
# !alias sh='yes | sh'
!yes | sh -c "$(curl -fsSL https://ftp.ncbi.nlm.nih.gov/entrez/entrezdirect/install-edirect.sh)"


Entrez Direct has been successfully downloaded and installed.


To activate EDirect for this terminal session, please execute the following:

export PATH=${HOME}/edirect:${PATH}



In [ ]:
!export PATH=${HOME}/edirect:${PATH}

In [ ]:
!esearch -db pubmed -query '"2023/01/01"[PDAT] : "2023/04/30"[PDAT]' | efetch -format abstract | tee articles_01.01_04.30.txt

/bin/bash: line 1: esearch: command not found


### Preparing the fetched data

In [2]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import pandas as pd
import re

In [3]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [4]:
# download data

# https://drive.google.com/file/d/13Cxu_rHEUrfxD-5d5CzW60z24_TUfIoF/view?usp=drive_link
download = drive.CreateFile({'id': '13Cxu_rHEUrfxD-5d5CzW60z24_TUfIoF'})
download.GetContentFile('articles_01.01_04.30.txt')

# https://drive.google.com/file/d/1-CrdSllQYvxtWiyWI6U3yPUt1JZTQgUF/view?usp=drive_link
download = drive.CreateFile({'id': '1-CrdSllQYvxtWiyWI6U3yPUt1JZTQgUF'})
download.GetContentFile('articles_05.01_06.30.txt')

# https://drive.google.com/file/d//view?usp=drive_link
download = drive.CreateFile({'id': '1ixHh--ZqLXcePBDycO-a5-bDXYh9KCnt'})
download.GetContentFile('articles_07.01_08.13.txt')

In [5]:
# load data and process data
""" all file names:
articles_01.01_04.30.txt
articles_05.01_06.30.txt
articles_07.01_08.13.txt
"""
file_names = ["articles_01.01_04.30", "articles_05.01_06.30", "articles_07.01_08.13"]
all_articles = []


for file_name in file_names
  with open(f"{file_name}.txt", 'r') as file:
    articles = []
    count = 0
    first_title = ""
    second_title = ""
    abstract = ""
    pubid = ""
    for line in file:
        line = line.rstrip()
        if line:
          new_article_pattern = re.compile("^\d*\.\s")
          if new_article_pattern.match(line):
            # save the last extracted article
            if len(abstract) > 10 and len(pubid) > 1:
              articles.append([pubid, first_title, second_title, abstract])

            count = 0
            first_title = ""
            second_title = ""
            abstract = ""
            pubid = ""

          if count == 0:
            first_title = first_title + " " + line
          elif count == 1:
            second_title = second_title + " " + line
          elif count == 4:
            abstract = abstract + " " + line

          # pubid_pattern = re.compile("PMID:\s\d*\s")
          # pubid_raw = pubid_pattern.match(line)
          try:
            result = re.match("PMID:\s\d*", line).group().split()
            pubid = result[-1].strip()
          except AttributeError:
            pass

          # print(line)
          # print("----------------------------------")
        else:
          # print("**********************")
          count += 1
    all_articles.append(articles)

In [6]:
all_articles_dfs = []
for articles in all_articles:
  articles_df = pd.DataFrame(articles, columns=['pubid', 'first_title', 'second_title', 'abstract'])
  all_articles_dfs.append(articles_df)
  print(articles_df)

,pubid,first_title,second_title,abstract
0,20301299,1. Heritable Thoracic Aortic Disease Overview.,"Milewicz DM(1), Cecchi AC(1).",The goals of this overview on heritable thora...
1,20301310,2. Arrhythmogenic Right Ventricular Cardiomyo...,"McNally E(1), MacLeod H(2), Dellefave-Castill...",The purpose of this overview is to increase t...
2,20301311,3. Nail-Patella Syndrome.,"Sweeney E(1), Hoover-Fong JE(2), McIntosh I(3).",CLINICAL CHARACTERISTICS: Nail-patella syndro...
3,20301331,4. Achondroplasia.,Legare JM(1).,CLINICAL CHARACTERISTICS: Achondroplasia is t...
4,20301352,5. Mitochondrial DNA-Associated Leigh Syndrom...,"Thorburn DR(1)(2), Rahman J(3), Rahman S(4).",CLINICAL CHARACTERISTICS: Mitochondrial DNA (...
...,...,...,...,...
318070,37577402,248. Hortic Res. 2023 May 22;10(7):uhad112. d...,Dopamine alleviates cadmium stress in apple t...,Dopamine has demonstrated promise as a stress...
318071,37577403,249. Hortic Res. 2023 May 19;10(7):uhad107. d...,Omics analyses in citrus reveal a possible ro...,Environmental changes derived from global war...
318072,37577404,250. Hortic Res. 2023 May 31;10(7):uhad115. d...,FvWRKY50 is an important gene that regulates ...,The WRKY transcription factors play important...
318073,37577405,251. Hortic Res. 2023 May 31;10(7):uhad109. d...,Unveiling the spatial distribution and molecu...,Salvia miltiorrhiza and S. grandifolia are ri...


In [7]:
# save data
""" all file names:
articles_01.01_04.30.txt
articles_05.01_06.30.txt
articles_07.01_08.13.txt
"""
for i, articles_df in enumerate(all_articles_dfs):
  articles_df.to_csv(f"{file_names[i]}.csv", sep='\t')